In [35]:
%%bigquery

BEGIN

CREATE SCHEMA sample_data;

create or replace table `sample_data.smiles` as
(
SELECT product_smiles as smiles,
from `nih-nci-cbiit-chem-prod.savi.plus` 
where rand() < 0.001
);

END

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 941.27query/s] 


""


In [30]:
%%bigquery

CREATE SCHEMA exports;

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 735.97query/s] 


""


In [38]:
%%bigquery 

create or replace table `exports.savi_fp_json` as
(
SELECT smiles,
`cheminformatics_test.rdkit_pattern_fingerprint_test`(smiles) as fingerprint_json
from `sample_data.smiles` 
)

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1369.49query/s]                        


""


In [ ]:
%%bigquery 

create or replace table `exports.savi_molecular_descriptors_json` as
(
SELECT smiles,
`cheminformatics_test.rdkit_molecular_descriptors`(smiles) as molecular_descriptors_json
from `sample_data.smiles` 
)

In [43]:
%%bigquery

create or replace table `exports.savi_fp` as
(
SELECT smiles,
FROM_HEX(JSON_VALUE(fingerprint_json, '$.fp_pattern_long_as_binary_hex'))  as fp_pattern_as_binary,
FROM_HEX(JSON_VALUE(fingerprint_json, '$.fp_morgan_as_binary_hex'))  as fp_morgan_as_binary,
CAST(JSON_VALUE(fingerprint_json, "$.num_carbon") as INT64) as num_carbon,
CAST(JSON_VALUE(fingerprint_json, "$.num_oxygen") as INT64) as num_oxygen,
CAST(JSON_VALUE(fingerprint_json, "$.num_nitrogen") as INT64) as num_nitrogen,
CAST(JSON_VALUE(fingerprint_json, "$.num_fluorine") as INT64) as num_fluorine,
CAST(JSON_VALUE(fingerprint_json, "$.num_sulfur") as INT64) as num_sulfur,
from `exports.savi_fp_json` where fingerprint_json != ""
)

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1477.39query/s]                        


""


In [45]:
%%bigquery

create or replace table `exports.savi_fp_clustered`
cluster by num_carbon, num_nitrogen, num_oxygen, num_fluorine
as 
SELECT * FROM `exports.savi_fp` 

Query complete after 0.00s: 100%|██████████| 5/5 [00:00<00:00, 2491.86query/s]                        


""


In [70]:
%%bigquery df

CALL `cheminformatics_test.substructure_search`("C[C@@H](CC(=C)C)C(=O)OCC1[C@@H]2COC[C@H]12");

Downloading: 100%|██████████| 1/1 [00:02<00:00,  2.20s/rows]


In [71]:
df

,smiles
0,C[C@@H](CC(=C)C)C(=O)OCC1[C@@H]2COC[C@H]12


In [72]:
%%bigquery df

CALL `cheminformatics_test.morgan_similarity`("C[C@@H](CC(=C)C)C(=O)OCC1[C@@H]2COC[C@H]12");

Downloading: 100%|██████████| 10/10 [00:01<00:00,  5.24rows/s]


In [73]:
df

,smiles
0,C[C@@H](CC(=C)C)C(=O)OCC1[C@@H]2COC[C@H]12
1,O(CC1[C@@H]2COC[C@H]12)C(C3CC(C)C3)=O
2,O(CC1CCC(=O)O1)C([C@@H](C)CC(=C)C)=O
3,O(CC1[C@@H]2COC[C@H]12)CCCCC(C)=O
4,C[C@@H]1CCC[C@H]1C(=O)OCC2[C@@H]3COC[C@H]23
5,NC(CC(F)(F)F)C(=O)OCC1[C@@H]2COC[C@H]12
6,O(CC1[C@@H]2COC[C@H]12)C(=O)C3CC[S](=O)(=O)C3
7,O(CC1[C@@H]2COC[C@H]12)C(CCOC3=CC=C(C)C=C3)=O
8,CC1(C)CCC=C1C(=O)OCC2[C@@H]3COC[C@H]23
9,O(CC1[C@@H]2COC[C@H]12)C(CC3=N[N](C(C)C)C=C3)=O
